In [5]:
import pandas as pd

# Specify the path to your Excel file
file_path = 'dedup_conversations.xlsx'

# Specify the sheet name or index. For example, 'Sheet1' or 0 for the first sheet
all_sheet_name = 'all'
# Read the specified sheet of the Excel file
df_all = pd.read_excel(file_path, sheet_name=all_sheet_name)


# Specify the sheet name or index. For example, 'Sheet1' or 0 for the first sheet
submitted_sheet_name = 'submitted'
# Read the specified sheet of the Excel file
df_submitted = pd.read_excel(file_path, sheet_name=submitted_sheet_name)

In [9]:
# Accessing the 'task_link' column from the df_submitted DataFrame
submitted_tasks = df_submitted['task_link']
# Creating a set from the 'task_link' column
submitted_tasks_set = set(submitted_tasks)

len(submitted_tasks)



300

In [39]:
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from openpyxl.worksheet.datavalidation import DataValidation
from copy import copy
from openpyxl.utils.cell import get_column_letter

# Load the workbook and the 'all' sheet
source_file_path = 'dedup_conversations.xlsx'
source_workbook = load_workbook(filename=source_file_path)
sheet_all = source_workbook['all']

# Create a new workbook for the split sheets
new_workbook = Workbook()
sheet_submitted = new_workbook.active
sheet_submitted.title = 'submitted'
sheet_unsubmitted = new_workbook.create_sheet('unsubmitted')

# Find the index of the 'task_link' column
task_link_index = None
for idx, cell in enumerate(sheet_all[1]):  # Assuming the first row contains headers
    if cell.value == 'task_link':
        task_link_index = idx
        break

if task_link_index is None:
    raise ValueError("The 'task_link' column was not found in the 'all' sheet.")

# Function to copy a row to a new sheet
def copy_row_to_sheet(row, target_sheet, row_num):
    for col_num, cell in enumerate(row, start=1):
        new_cell = target_sheet.cell(row=row_num, column=col_num)
        new_cell.value = cell.value
        if cell.hyperlink:  # Copy hyperlink if present
            new_cell.hyperlink = copy(cell.hyperlink)
        if cell.has_style:  # Copy style if present
            new_cell.font = copy(cell.font)
            new_cell.border = copy(cell.border)
            new_cell.fill = copy(cell.fill)
            new_cell.number_format = copy(cell.number_format)
            new_cell.protection = copy(cell.protection)
            new_cell.alignment = copy(cell.alignment)

# Copy the header row to both new sheets
header_row = next(sheet_all.iter_rows(min_row=1, max_row=1))
copy_row_to_sheet(header_row, sheet_submitted, 1)
copy_row_to_sheet(header_row, sheet_unsubmitted, 1)

# Copy data validation for drop-down menus and checkboxes
for dv in sheet_all.data_validations.dataValidation:
    new_dv = copy(dv)
    # Adjust the range for the new sheet
    new_ranges = []
    for range_ in dv.ranges:
        # Assuming the data validation is applied to the entire column
        col_letter = get_column_letter(range_.min_col)
        new_range_str = f"{col_letter}2:{col_letter}{sheet_unsubmitted.max_row + 1}"
        new_ranges.append(new_range_str)
    new_dv.ranges = new_ranges
    sheet_submitted.add_data_validation(new_dv)
    sheet_unsubmitted.add_data_validation(new_dv)

# Iterate over the 'all' sheet and copy rows to the corresponding new sheet
submitted_row_num = 2
unsubmitted_row_num = 2
for row in sheet_all.iter_rows(min_row=2):
    task_link = row[task_link_index].value
    if isinstance(task_link, str) and task_link.startswith('https'):
        if task_link in submitted_tasks_set:
            copy_row_to_sheet(row, sheet_submitted, submitted_row_num)
            submitted_row_num += 1
        else:
            copy_row_to_sheet(row, sheet_unsubmitted, unsubmitted_row_num)
            unsubmitted_row_num += 1

# Save the new workbook
new_file_path = 'split_dedup_conversations.xlsx'
new_workbook.save(filename=new_file_path)